# ALICE Masterclass R$_{AA}$

    Load necessary packages

In [ ]:
import ALICE_RAA_Tools as at
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import numpy as np

List of available centrality classes:

    0-5%, 0-10%, 5-10%, 10-20%, 20-30%, 30-40%, 40-50%, 50-60%, 60-70%, 70-80%, 80-90%

__ATTENTION:__
Here you can modify, add, or delete centrality classes. But be careful. The centrality class 70-80% serves as reference for R$_{cp}$. Therfore, it shouldn't be deleted.
    

In [ ]:
listCentralities = [(60,70), (70,80)]

    Create a Dictionary

In [ ]:
dictCentralities = at.create_dictionary(listCentralities)


# 1. Event Analysis

### 1.1 Read the events

    Count the events

In [ ]:
event_lines = at.count_lines('event_information.csv')
print("Number of events: '" + str(event_lines) + "'!")

    Read event information from CSV file

In [ ]:
df_events = pd.read_csv("event_information.csv", header=None, names=['eventMult', 'eventCent'])

In [ ]:
print(df_events.head(10))

### 1.2 Process the data

    Add columns to DataFrame for each centrality class

In [ ]:
for key in dictCentralities:
    df_events[key] = False

print(df_events.head(10))

    Set the centrality information und count the events for each centrality class

In [ ]:
dictEventsCent = {}
for row in df_events.iterrows():
    keys = at.find_centralities(dictCentralities, row[1]['eventCent'])
    if keys:
        for cent in keys:
            df_events.loc[row[0], cent] = True
            if cent in dictEventsCent:
                dictEventsCent[cent] += 1
            else:
                dictEventsCent[cent] = 1

In [ ]:
print(df_events.head(10))
print(dictEventsCent)

### 1.3 Output of the results

    Create and plot particle-multiplicity histograms

In [ ]:
hTPC_0_100, bins = at.create_event_histos(df_events['eventMult'])
hTPC_60_70, bins = at.create_event_histos(df_events[df_events['60-70'] == True]['eventMult'])
hTPC_70_80, bins = at.create_event_histos(df_events[df_events['70-80'] == True]['eventMult'])

In [ ]:
hTPC_0_100_err = np.sqrt(hTPC_0_100)
hTPC_60_70_err = np.sqrt(hTPC_60_70)
hTPC_70_80_err = np.sqrt(hTPC_70_80)

In [ ]:
plt.errorbar(bins[:-1], hTPC_0_100, yerr=hTPC_0_100_err, fmt='g+', label='0-100%')
plt.errorbar(bins[:-1], hTPC_60_70, yerr=hTPC_60_70_err, fmt='b+', label='60-70%')
plt.errorbar(bins[:-1], hTPC_70_80, yerr=hTPC_70_80_err, fmt='r+', label='70-80%')
#for the 'fmt' option you find useful information here:
#https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.plot.html#matplotlib.pyplot.plot

plt.xlabel('Number of TPC tracks')
plt.ylabel('Counts')
plt.ylim(1,2E4)
plt.yscale('log')

plt.legend(loc='upper right')
plt.savefig('TPC_tracks.pdf')

    2D plot: charged-particle track multiplicity vs. centrality

In [ ]:
plt.hist2d(df_events['eventMult'],df_events['eventCent'], bins=(200,100), range=[[0, 2000], [0, 100]], cmap=plt.cm.jet, norm=mpl.colors.LogNorm())

plt.colorbar()
plt.xlabel('Number of TPC tracks')
plt.ylabel('Centrality')
plt.savefig('TPC_tracks_2D.pdf')

# 2. Charged-particle tracks und momentum spectra

    List of available centrality classes:
 
     0-5%, 0-10%, 5-10%, 10-20%, 20-30%, 30-40%, 40-50%, 50-60%, 60-70%, 70-80%, 80-90%

## 2.1 Read the Pb-Pb data

In [ ]:
print("Number of tracks: '38042122'! ~600 MB file (unpacked)")

In [ ]:
dictMomCent = {}
for key in dictCentralities:
    dictMomCent[key] = []

In [ ]:
df_tracks = pd.read_pickle("./track_info.pkl", 'bz2').to_numpy()

### 2.2 Sort according to centrality class

In [ ]:
for line in df_tracks:
    keys = at.find_centralities(dictCentralities, line[1])
    if keys:
        for key in keys:
            dictMomCent[key].append(line[0])

### 2.3 Creation of histograms

In [ ]:
binsPt = at.get_bins()
binWidth = at.get_bin_width(binsPt)
x_bin_width = np.asarray(binWidth)/2

In [ ]:
hPbPb_60_70, _ = np.histogram(dictMomCent['60-70'], binsPt)
hPbPb_60_70_err = np.sqrt(hPbPb_60_70) / dictEventsCent['60-70']
hPbPb_60_70 = hPbPb_60_70 / binWidth
hPbPb_60_70 = hPbPb_60_70 / dictEventsCent['60-70']

hPbPb_70_80, _ = np.histogram(dictMomCent['70-80'], binsPt)
# Spezial case for 70-80%: one bin has zero counts - problematic for division later on - therefore: replace 0 with 1
hPbPb_70_80 = np.where(hPbPb_70_80==0,1,hPbPb_70_80)
hPbPb_70_80_err = np.sqrt(hPbPb_70_80) / dictEventsCent['70-80']
hPbPb_70_80 = hPbPb_70_80 / binWidth
hPbPb_70_80 = hPbPb_70_80 / dictEventsCent['70-80']

### 2.4 Read the proton-proton reference

In [ ]:
pp = np.genfromtxt('pp_reference.dat')
print(pp)
pp_err = np.divide(pp,10) #Assumption: 10% uncertainty on pp data

### 2.5 Plot transverse-momentum spectra of charged particles

### 2.6 Define centrality classes

Define number of binary collisions for each centrality class

In [ ]:
dictNColl = {'0-5': 1686.87,
             '0-10': 1502.7,
             '5-10': 1319.89,
             '10-20': 923.89,
             '20-30': 558.68,
             '30-40': 321.20,
             '40-50': 171.67,
             '50-60': 85.13,
             '60-70': 38.51,
             '70-80': 15.78,
             '80-90': 6.32,
             'pp':1.
            }

print(dictNColl)

### 2.7 Calculate and plot R$_\text{AA}$

### 2.8 Extra task: Calculate and plot R$_{\text{CP}}$